In [4]:
import pandas as pd

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource

from bokeh.models import HoverTool
from bokeh.transform import factor_cmap
from bokeh.palettes import Category10

In [5]:
output_notebook()

Loading BokehJS ...

In [6]:
# Specify your own file path... Reading the data from excel file
dfAuto = pd.read_excel('data/Auto_Insu.xlsx')

In [7]:
# Converting to string
dfAuto.NumCylinders = dfAuto.NumCylinders.astype(str)

In [8]:
autoCDS = ColumnDataSource(dfAuto)

In [9]:
myCDSPlot = figure(height=400, width=400,
                  title='HP Vs City MPG',
                  x_axis_label='Horse Power',
                  y_axis_label='City-MPG')

In [10]:
scatMap = factor_cmap('NumCylinders', factors=dfAuto.NumCylinders.unique(), palette=Category10[8])

In [11]:
myCDSPlot.circle(x='HP', y='City_MPG',
                color=scatMap,
                # legend=,
                size=3,
                source=autoCDS)

myCDSPlot.x_range.start = 0
myCDSPlot.y_range.start = 0

# Linked Interactions for Panning and Brushing

In [12]:
from bokeh.layouts import Row
from bokeh.models import BoxSelectTool

In [13]:
# Chart 2 for linked panning/brushing HP Vs. Highway-MPG
myCDSPlot2 = figure(height=400, width=400,
                  title='HP Vs Highway MPG',
                  x_axis_label='Horse Power',
                  y_axis_label='Highway MPG',
                   x_range=myCDSPlot.x_range,
                   y_range=myCDSPlot.y_range)

myCDSPlot2.square(x='HP', y='Highway_MPG',
                 color=scatMap,
                 size=3,
                 source=autoCDS)

myCDSPlot.add_tools(BoxSelectTool())
myCDSPlot2.add_tools(BoxSelectTool())

In [14]:
show(Row(myCDSPlot, myCDSPlot2))

# Creating multi-valued clustered bar charts with interactive legends

In [15]:
# Accessing data from CDS
# autoCDS.data['Make']

In [16]:
dfMakeMPG = dfAuto.groupby(['Make'])['City_MPG', 'Highway_MPG','Price'].mean().reset_index().sort_values(by='Make')
# https://realpython.com/python-data-visualization-bokeh/#using-the-columndatasource-object
dfMakeMPG.head()

,Make,City_MPG,Highway_MPG,Price
0,alfa_romero,20.333333,26.666667,15498.333333
1,audi,18.857143,24.142857,18942.000000
2,bmw,19.375000,25.375000,26118.750000
3,chevrolet,41.000000,46.333333,6007.000000
4,dodge,28.000000,34.111111,7875.444444


In [17]:
cdsMake = ColumnDataSource(dfMakeMPG)

In [18]:
cdsMake.column_names

['Make', 'City_MPG', 'Highway_MPG', 'Price', 'index']

In [19]:
from bokeh.transform import dodge
from bokeh.core.properties import value

In [20]:
# Creating categorical clustered column chart using DODGE function
autoBar = figure(height=400, width=800,
                 # Unique to make sure each category appears just once    
                 x_range=dfMakeMPG['Make'].unique())

In [21]:
autoBar.vbar(x=dodge('Make', 0.0, range=autoBar.x_range), 
             top='City_MPG', 
             width=.2,
             color = 'green',
             line_color='white',
             legend=value("City_MPG"),
             source=cdsMake,
             muted_color='green', muted_alpha=0.2)

autoBar.vbar(x=dodge('Make', 0.3, range=autoBar.x_range), 
             top='Highway_MPG', 
             width=.2,
             color='red',
             line_color='white',
             legend=value("Highway_MPG"),
             source=cdsMake,
             muted_color='red',muted_alpha=0.2)

autoBar.legend.location = "top_right"
#autoBar.legend.click_policy="hide"
autoBar.legend.click_policy="mute"

myHover = HoverTool(tooltips=[("Avg Price",'@Price{($ 0.00)}')])

autoBar.add_tools(myHover)

#autoBar.x_range.range_padding = .1
autoBar.y_range.range_padding = None
autoBar.y_range.end=50

autoBar.xaxis.major_label_orientation =45

show(autoBar)

# Nested categorical bar chart with pandas GROUPBY and Hover Tool

In [22]:
# Converting to string
dfAuto.NumCylinders = dfAuto.NumCylinders.astype(str)

# Creating grouby object
dfMakeCylMPG = dfAuto.groupby(['NumCylinders','Make'])
dfMakeCylMPG.describe()

Bore                                               \
                           count      mean           std   min     25%    50%   
NumCylinders Make                                                               
12           jaguar          1.0  3.540000           NaN  3.54  3.5400  3.540   
2            mazda           4.0  3.290000  8.000000e-02  3.17  3.2900  3.330   
3            chevrolet       1.0  2.910000           NaN  2.91  2.9100  2.910   
4            alfa_romero     2.0  3.470000  0.000000e+00  3.47  3.4700  3.470   
             audi            1.0  3.190000           NaN  3.19  3.1900  3.190   
             bmw             2.0  3.500000  0.000000e+00  3.50  3.5000  3.500   
             chevrolet       2.0  3.030000  0.000000e+00  3.03  3.0300  3.030   
             dodge           9.0  3.094444  2.240598e-01  2.97  2.9700  2.970   
             honda          13.0  3.021538  1.238175e-01  2.91  2.9100  2.920   
             isuzu           4.0  3.200000  2.023199e-01  3.03  3.0300  3.170   
             mazda          13.0  3.286923  2.263620e-01  3.03  3.0300  3.390   
             mercury         1.0  3.780000           NaN  3.78  3.7800  3.780   
             mitsubishi     13.0  3.250769  2.368003e-01  2.97  3.0300  3.170   
             nissan         12.0  3.166667  8.896714e-02  2.99  3.1500  3.150   
             peugot         11.0  3.582727  1.203404e-01  3.46  3.4600  3.610   
             plymouth        7.0  3.121429  2.489597e-01  2.97  2.9700  2.970   
             porsche         1.0  3.940000           NaN  3.94  3.9400  3.940   
             renault         2.0  3.460000  0.000000e+00  3.46  3.4600  3.460   
             saab            6.0  3.373333  4.082483e-01  2.54  3.5400  3.540   
             subaru         12.0  3.620000  4.638361e-16  3.62  3.6200  3.620   
             toyota         28.0  3.281429  1.995126e-01  3.05  3.1900  3.240   
             volkswagen     11.0  3.124545  9.081450e-02  3.01  3.0100  3.190   
             volvo           9.0  3.744444  7.055337e-02  3.62  3.7800  3.780   
5            audi            6.0  3.170000  3.098387e-02  3.13  3.1450  3.190   
             mercedes_benz   4.0  3.580000  0.000000e+00  3.58  3.5800  3.580   
             volkswagen      1.0  3.190000           NaN  3.19  3.1900  3.190   
6            alfa_romero     1.0  2.680000           NaN  2.68  2.6800  2.680   
             bmw             6.0  3.465000  1.697940e-01  3.31  3.3100  3.465   
             jaguar          2.0  3.630000  0.000000e+00  3.63  3.6300  3.630   
             nissan          6.0  3.430000  0.000000e+00  3.43  3.4300  3.430   
             porsche         3.0  3.740000  0.000000e+00  3.74  3.7400  3.740   
             toyota          4.0  3.270000  0.000000e+00  3.27  3.2700  3.270   
             volvo           2.0  3.295000  4.030509e-01  3.01  3.1525  3.295   
8            mercedes_benz   4.0  3.630000  1.962991e-01  3.46  3.4600  3.630   
             porsche         1.0  3.940000           NaN  3.94  3.9400  3.940   

                                         City_MPG             ...  Wheel_Base  \
                               75%   max    count       mean  ...         75%   
NumCylinders Make                                             ...               
12           jaguar         3.5400  3.54      1.0  13.000000  ...     102.000   
2            mazda          3.3300  3.33      4.0  16.750000  ...      95.300   
3            chevrolet      2.9100  2.91      1.0  47.000000  ...      88.400   
4            alfa_romero    3.4700  3.47      2.0  21.000000  ...      88.600   
             audi           3.1900  3.19      1.0  24.000000  ...      99.800   
             bmw            3.5000  3.50      2.0  23.000000  ...     101.200   
             chevrolet      3.0300  3.03      2.0  38.000000  ...      94.500   
             dodge          3.0300  3.60      9.0  28.000000  ...      93.700   
             honda          3.1500  3.15     13.0  30.3846

In [23]:
autoNest =  figure(height=300,#width=800,
                  x_range=dfMakeCylMPG)

myHover = HoverTool(tooltips=[("Curb-Weight_mean",'@Width_max'),
                             ("Avg Price",'@Price_mean{($ 0.00)}')])

autoNest.add_tools(myHover)
autoNest.xaxis.major_label_orientation = 1.3
autoNest.xgrid.grid_line_color=None

In [24]:
myColor = factor_cmap('NumCylinders_Make', palette=Category10[8], factors=sorted(dfAuto.NumCylinders.unique()), end=1)
autoNest.vbar(x='NumCylinders_Make', top='City_MPG_mean', width=.5, color=myColor, source=dfMakeCylMPG)

bokeh.models.renderers.GlyphRenderer(
    id='d8f70341-257d-4758-9978-a064377de9b8',
    data_source=bokeh.models.sources.ColumnDataSource(
        id='eda402d4-2bd8-49eb-888f-43854a7207d0',
        callback=None,
        data={'Stroke_max': {'__ndarray__': 'FK5H4XoUBkDD9Shcj8ILQD0K16NwPQhAcT0K16NwBUAzMzMzMzMLQGZmZmZmZgZA4XoUrkfhCEAzMzMzMzMPQKRwPQrXowxA16NwPQrXCUAfhetRuB4NQPYoXI/C9QhA4XoUrkfhDkDD9Shcj8ILQClcj8L1KAxA4XoUrkfhDkDhehSuR+EIQDMzMzMzMw9Aj8L1KFyPCEAfhetRuB4FQFK4HoXrUQxAMzMzMzMzC0AzMzMzMzMJQDMzMzMzMwtAH4XrUbgeDUAzMzMzMzMLQMP1KFyPwgtAH4XrUbgeC0CuR+F6FK4QQClcj8L1KApAMzMzMzMzB0DNzMzMzMwKQDMzMzMzMwtAzczMzMzMCkDhehSuR+EIQA==',
          'shape': (35,),
          'dtype': 'float64'},
         'Bore_75%': {'__ndarray__': 'UrgehetRDECkcD0K16MKQEjhehSuRwdAw/UoXI/CC0CF61G4HoUJQAAAAAAAAAxAPQrXo3A9CEA9CtejcD0IQDMzMzMzMwlAuB6F61G4CkAfhetRuB4LQD0K16NwPQ5AzczMzMzMCkAzMzMzMzMJQJqZmZmZmQ1AhetRuB6FCUCF61G4HoUPQK5H4XoUrgtAUrgehetRDED2KFyPwvUMQHsUrkfhegpAhetRuB6FCUA9CtejcD0OQIXrUbgehQlApHA9CtejDECF61G4HoUJQHE9CtejcAVA9ihcj8L1DEAK16NwPQoNQHE9CtejcAtA7FG4HoXrDUApXI/C9SgKQAAAAAAAgAtAZmZmZmZmDkCF61G4HoUPQA==',
          'shape': (35,),
          'dtype': 'float64'},
         'Peak_RPM_75%': {'__ndarray__': 'AAAAAACIs0AAAAAAAHC3QAAAAAAA7LNAAAAAAACIs0AAAAAAAHy1QAAAAAAAqLZAAAAAAAAYtUAAAAAAAHy1QAAAAAAAcLdAAAAAAAAYtUAAAAAAAIizQAAAAAAAiLNAAAAAAAB8tUAAAAAAAFC0QAAAAAAAiLNAAAAAAAB8tUAAAAAAAHy1QAAAAAAA7LNAAAAAAIA9tUAAAAAAANmyQAAAAAAAwLJAAAAAAAB8tUAAAAAAABi1QAAAAAAAfLVAAAAAAAD+sEAAAAAAAHy1QAAAAAAAiLNAAAAAAAAYtUAAAAAAAI6yQAAAAAAAULRAAAAAAAAMt0AAAAAAAFC0QAAAAAAAzbRAAAAAAACOskAAAAAAAHa2QA==',
          'shape': (35,),
          'dtype': 'float64'},
         'Compression_Ratio_count': {'__ndarray__': 'AAAAAAAA8D8AAAAAAAAQQAAAAAAAAPA/AAAAAAAAAEAAAAAAAADwPwAAAAAAAABAAAAAAAAAAEAAAAAAAAAiQAAAAAAAACpAAAAAAAAAEEAAAAAAAAAqQAAAAAAAAPA/AAAAAAAAKkAAAAAAAAAoQAAAAAAAACZAAAAAAAAAHEAAAAAAAADwPwAAAAAAAABAAAAAAAAAGEAAAAAAAAAoQAAAAAAAADxAAAAAAAAAJkAAAAAAAAAiQAAAAAAAABhAAAAAAAAAEEAAAAAAAADwPwAAAAAAAPA/AAAAAAAAGEAAAAAAAAAAQAAAAAAAABhAAAAAAAAACEAAAAAAAAAQQAAAAAAAAABAAAAAAAAAEEAAAAAAAADwPw==',
          'shape': (35,),
          'dtype': 'float64'},
         'Height_25%': {'__ndarray__': 'ZmZmZmbmR0DNzMzMzMxIQJqZmZmZmUpAZmZmZmZmSEBmZmZmZiZLQGZmZmZmJktAAAAAAAAASkDNzMzMzExJQM3MzMzMTEpAzczMzMzsSUDNzMzMzAxLQGZmZmZmZktAmpmZmZkZSUAAAAAAAMBKQJqZmZmZWUxAmpmZmZlZSUCamZmZmRlJQGZmZmZm1klAzczMzMwMTEAAAAAAAHBKQM3MzMzMTEpAzczMzMysS0AAAAAAAMBLQDQzMzMzs0pAmZmZmZn5S0DNzMzMzIxLQDMzMzMzM0pAZmZmZmYmS0BmZmZmZmZKQJqZmZmZ2UhAzczMzMzMSUAAAAAAAABKQAAAAAAAwEtAAAAAAAAgS0AAAAAAAEBJQA==',
          'shape': (35,),
          'dtype': 'float64'},
         'Height_min': {'__ndarray__': 'ZmZmZmbmR0DNzMzMzMxIQJqZmZmZmUpAZmZmZmZmSEBmZmZmZiZLQGZmZmZmJktAAAAAAAAASkCamZmZmRlJQGZmZmZmZklAMzMzMzOzSUCamZmZmdlKQGZmZmZmZktAMzMzMzOzSEBmZmZmZqZKQAAAAAAAAExAmpmZmZkZSUCamZmZmRlJQAAAAAAAQElAzczMzMwMTEAAAAAAAEBKQAAAAAAAAEpAMzMzMzOzSUAAAAAAAMBLQAAAAAAAAEpAMzMzMzNzS0DNzMzMzIxLQDMzMzMzM0pAmpmZmZnZSkBmZmZmZmZKQJqZmZmZ2UhAzczMzMzMSUAAAAAAAABKQAAAAAAAwEtAZmZmZmZmSUAAAAAAAEBJQA==',
          'shape': (35,),
          'dtype': 'float64'},
         'Width_25%': {'__ndarray__': 'ZmZmZmamUUDNzMzMzGxQQGZmZmZmJk5AZmZmZmYGUEDNzMzMzIxQQDMzMzMzM1BAzczMzMzMT0BmZmZmZuZPQDMzMzMz809ANDMzMzOTT0DNzMzMzAxQQAAAAAAAAFFAmpmZmZkZUEBmZmZmZuZPQJqZmZmZGVFAZmZmZmbmT0AzMzMzMxNRQJqZmZmZoVBAAAAAAACgUEAAAAAAAEBQQAAAAAAAAFBAAAAAAABgUEDNzMzMzMxQQJqZmZmZsVBAMzMzMzOTUUCamZmZmblQQAAAAAAAYFBAzMzMzMxUUEBmZmZmZmZRQAAAAAAAoFBAAAAAAABAUEAAAAAAAKBQQJqZmZmZOVFAmpmZmZnZUUAzMzMzMxNSQA==',
          'shape': (35,),
          'dtype': 'float64'},
         'Curb_Weight_min': {'__ndarray__': 'AAAAAADcrkAAAAAAAJiiQAAAAAAAQJdAAAAAAADoo0AAAAAAAEKiQAAAAAAAtqJAAAAAAABInUAAAAAAAFCdQAAAAAAAxJpAAAAAAABInUAAAAAAAIidQAAAAAAAvKZAAAAAAAD4nUAAAAAAAISdQAAAAAAAmKdAAAAAAAD4nUAAAAAAALSlQAAAAAAAOKNAAAAAAADEpEAAAAAAAASgQAAAAAAABJ9AAAAAAABCoUAAAAAAAMCmQAAAAAAAlqNAAAAAAABOq0AAAAAAAMqkQAAAAAAADqZAAAAAAAAspUAAAAAAAMSvQAAAAAAA6KdAAAAAAACIpUAAAAAAAECnQAAAAAAAiKdAAAAAAADKrEAAAAAAAEyqQA==',
          'sh

In [25]:
show(autoNest)

# Interactive Legends with Line Chart

In [26]:
dfStock = pd.read_excel('data/StocksClosingPrices.xlsx')

In [27]:
cdsStock = ColumnDataSource(dfStock)

In [28]:
dfStock.head()

,Date,Amazon,Google,IBM
0,2013-02-01,265.000000,385.294037,205.179993
1,2013-02-04,259.980011,377.057617,203.789993
2,2013-02-05,266.890015,380.395905,202.789993
3,2013-02-06,262.220001,382.596588,201.020004
4,2013-02-07,260.230011,384.474365,199.740005


In [29]:
from bokeh.models import NumeralTickFormatter

In [34]:
myLine = figure(height=400,width=500,
               title="Interactive Legend with Line chart",
               x_axis_label='Time',
               y_axis_label='Stock Price',
               x_axis_type='datetime'
               )

myLine.y_range.start=0
myLine.yaxis.formatter=NumeralTickFormatter(format='$0')

In [35]:
myLine.line(x='Date',y='Amazon',legend=value('Amazon'),muted_color='blue', muted_alpha=0.1, color='blue', source=cdsStock)
myLine.line(x='Date',y='Google',legend=value('Google'),muted_color='red', muted_alpha=0.1, color='red', source=cdsStock)

myLine.legend.location = "top_right"
myLine.legend.click_policy='mute'

show(myLine)